### Import des bibliothéques: 

In [1]:
import numpy as np 
from mip import Model, xsum, maximize 
import math

### Classe MCTSNode: 
- La classe MCTSNode représente un nœud dans l'arbre de décision utilisé par l'algorithme de Monte Carlo Tree Search (MCTS).

- Chaque nœud a un état ("state") qui représente une alternative (par exemple, une option que le décideur pourrait choisir).
- Chaque nœud a une référence à son nœud parent ("parent"), permettant de remonter dans l'arbre.
- Les nœuds peuvent avoir plusieurs enfants ("children"), représentant des alternatives qui peuvent être explorées à partir de cet état.
- "visits" compte combien de fois ce nœud a été exploré, ce qui aide à évaluer son utilité.
- "value" accumule les résultats des simulations pour ce nœud, indiquant la qualité de l'alternative qu'il représente.

In [2]:
# Classe représentant un nœud de l'arbre MCTS
class MCTSNode:
    def __init__(self, state, parent=None):
        self.state = state  # L'état ou l'alternative
        self.parent = parent  # Parent du nœud
        self.children = []  # Les enfants du nœud
        self.visits = 0  # Nombre de visites de ce nœud
        self.value = 0  # Valeur cumulée du nœud (par exemple, regret ou score)
    
    def add_child(self, child_state):
        """Ajoute un enfant à ce nœud."""
        child_node = MCTSNode(state=child_state, parent=self)
        self.children.append(child_node)
        return child_node

### 1. Fonction M_Omega

In [3]:
# Fonction pour calculer le modèle de décision avec les poids
def M_omega(x, omega):
    return np.dot(x, omega)

- Calcule l'évaluation d'une solution 'x' avec un vecteur de poids 'omega'.
    
    Args:
    - x (array-like): Vecteur représentant une solution avec ses scores pour chaque critère.
    - omega (array-like): Vecteur représentant les poids associés à chaque critère.
    
    Returns:
    - float: La somme pondérée des scores de 'x' selon les poids 'omega'.

### 2. Fonction Query

In [4]:
# Fonction pour poser une question au décideur (simulée aléatoirement ici)
def Query(x, y):
    print(f"Comparaison entre {x} et {y}")
    return x if np.random.rand() > 0.5 else y

 - Pose une question au décideur pour comparer deux solutions 'x' et 'y'.
    La réponse est simulée par un choix aléatoire.
    
    Args:
    - x (array-like): Première solution à comparer.
    - y (array-like): Seconde solution à comparer.
    
    Returns:
    - array-like: La solution préférée entre 'x' et 'y'.

#### Résolution du probléme linéaire pour calculer PMR en utilisant MIP

La fonction PMR calcule le regret maximum par paires entre deux solutions x et y, en tenant compte des poids admissibles définis dans 
Ω. Le regret est la différence entre les évaluations des deux solutions, maximisée pour le pire ensemble de poids possible dans le polytope Ω. 

- Modélisation du problème : On modélise cela comme un problème de programmation linéaire où on veut maximiser la différence entre l’évaluation de y et de x en fonction des poids w (qui sont les variables de décision dans le modèle).

- On introduit une variable w pour chaque critère (chaque dimension du problème). Ces variables représentent les poids wi associés à chaque critère, qui doivent être trouvés pour maximiser le regret. Ces poids doivent respecter des contraintes de normalisation (somme égale à 1) et les autres contraintes définies dans Ω.

- Ajout des contraintes de Ω: Ω est un polytope défini par un ensemble de contraintes linéaires. Chaque contrainte dans Ω a la forme d’une équation linéaire qui doit être respectée pour que les poids soient admissibles. Dans le code, on itère sur toutes les contraintes définies dans Ω et on les ajoute au modèle.

-L’objectif est de maximiser la différence entre l'évaluation de y et x, c'est-à-dire maximiser Mw(y)-Mw(x), où Mw(.) est l’évaluation pondérée d’une solution par rapport aux poids w. Cela revient à maximiser la somme pondérée des différences entre y[i] et x[i].

### 3. Fonction PMR

In [5]:
# Fonction pour calculer le regret maximum par paires en tenant compte des contraintes de m
def PMR(x, y, m):  
    m_crit = len(x)  # Nombre de critères

    # Variables de décision : poids w_i pour chaque critère
    w = [m.add_var(lb=0) for _ in range(m_crit)]
    
    # Contrainte : somme des poids w_i = 1 (normalisation des poids)
    m += xsum(w[i] for i in range(m_crit)) == 1

    # Objectif : maximiser la différence pondérée entre 'y' et 'x'
    m.objective = maximize(xsum(w[i] * (y[i] - x[i]) for i in range(m_crit)))
    
    # Résoudre le problème
    m.optimize()
    
    # Obtenir le regret maximal (valeur de la fonction objectif)
    return m.objective_value

 - Calcule le regret maximum par paires (PMR) entre deux solutions 'x' et 'y'.
    Utilise la programmation linéaire pour maximiser le regret sous les contraintes définies dans 'Omega'.
    
    Args:
    - x (array-like): Première solution.
    - y (array-like): Seconde solution.
    - m (Model): Modèle MIP dans lequel les contraintes et l'objectif seront définis.
    
    Returns:
    - float: La valeur du regret maximum entre 'x' et 'y', optimisée avec le modèle MIP.

### 4. Fonction Update : Mise à jour du polytope Ω

La fonction Update a pour but de mettre à jour le polytope Ω, qui représente l'ensemble des poids possibles, en fonction des réponses du décideur à une question de préférence entre deux solutions x et y.

- Décision du décideur : Le décideur est interrogé pour savoir s'il préfère x ou y. En fonction de sa réponse, on sait que le modèle de décision sous-jacent doit respecter certaines relations linéaires entre les poids w:
    * Si x est préféré à y, cela signifie que la somme pondérée des critères pour x doit être plus grande ou égale à celle de y:         Mw(x) >= Mw(y).
    * Sinon, l'inverse.

- Ici, les coefficients de la contrainte sont la différence entre les vecteurs x et y. Le côté droit de l'inégalité est 0 car on compare directement les évaluations pondérées.

- Mise à jour du polytope Ω : Une nouvelle contrainte est ajoutée au polytope Ω à chaque nouvelle réponse. Cette contrainte réduit l'espace des poids admissibles en excluant ceux qui ne respectent pas la préférence du décideur. Cette mise à jour affine la connaissance des préférences du décideur.

In [6]:
# Fonction pour mettre à jour le polytope des poids possibles dans m
def Update(m, answer, x, y):
    """Met à jour le modèle MIP 'm' avec une nouvelle contrainte en fonction de la réponse du décideur."""
    if np.array_equal(answer, x):
        # x est préféré à y -> ajouter la contrainte M_omega(x) >= M_omega(y)
        new_constraint = xsum((x[i] - y[i]) * m.var_by_name(f"w[{i}]") for i in range(len(x))) >= 0
    else:
        # y est préféré à x -> ajouter la contrainte M_omega(y) >= M_omega(x)
        new_constraint = xsum((y[i] - x[i]) * m.var_by_name(f"w[{i}]") for i in range(len(x))) >= 0
    
    # Ajouter la nouvelle contrainte au modèle
    m += new_constraint
    
    return m

 - Met à jour le modèle MIP 'm' avec une nouvelle contrainte en fonction de la réponse du décideur.
    Si 'x' est préféré à 'y', on ajoute une contrainte qui reflète cette préférence, et inversement.

    Args:
    - m (Model): Modèle MIP à mettre à jour.
    - answer (array): L'alternative préférée par le décideur (résultat de la fonction 'Query').
    - x (array): Première alternative comparée.
    - y (array): Deuxième alternative comparée.

    Returns:
    - Model: Le modèle MIP mis à jour avec la nouvelle contrainte.

### Fonctions MR et mMR:

In [7]:
# Fonction pour calculer le regret maximum (MR)
def MR(x, X, m):
    """Calcule le regret maximum pour une alternative donnée 'x' par rapport à toutes les autres alternatives dans 'X'."""
    return max(PMR(x, y, m) for y in X if not np.array_equal(x, y))



-  Calcule le regret maximum pour une alternative donnée 'x' par rapport à toutes les autres alternatives dans 'X'.

    Args:
    - x (array): L'alternative pour laquelle on veut calculer le regret.
    - X (array of arrays): Ensemble d'alternatives à comparer avec 'x'.
    - m (Model): Modèle MIP utilisé pour optimiser les regrets.

    Returns:
    - float: Le regret maximum pour 'x' par rapport aux autres alternatives dans 'X'.

In [8]:
# Fonction pour calculer le regret minimax (mMR) et obtenir les solutions x_star et y_star
def mMR(X, m):
    """Calcule le regret minimax, c'est-à-dire l'alternative qui minimise son regret maximum par rapport aux autres."""
    x_star = min(X, key=lambda x: MR(x, X, m))
    y_star = max(X, key=lambda y: PMR(x_star, y, m))
    max_regret = MR(x_star, X, m)
    return max_regret, x_star, y_star



 - Calcule le regret minimax, c'est-à-dire l'alternative qui minimise son regret maximum par rapport aux autres.

    Args:
    - X (array of arrays): Ensemble d'alternatives.
    - m (Model): Modèle MIP utilisé pour optimiser les regrets.

    Returns:
    - tuple: (max_regret, x_star, y_star)
        - max_regret (float): Le regret minimax pour 'x_star'.
        - x_star (array): L'alternative qui minimise le regret maximum.
        - y_star (array): L'alternative qui maximise le regret par rapport à 'x_star'.

### 5. Fonction uct_value

- La fonction uct_value calcule la valeur UCT pour un nœud donné afin de guider la sélection.

- Si le nœud n'a pas été visité, il est priorisé pour l'exploration.
- La formule combine le score moyen du nœud (valeur par nombre de visites) et un terme d'exploration qui favorise les nœuds moins explorés.
- Cela aide à trouver un équilibre entre explorer de nouvelles alternatives et exploiter celles déjà connues.

In [9]:
def uct_value(node, exploration_param=1.41):
    if node.visits == 0:
        return float('inf')  # Favoriser l'exploration des nœuds non visités
    return node.value / node.visits + exploration_param * math.sqrt(math.log(node.parent.visits) / node.visits)


- Calcule la valeur UCT (Upper Confidence Bound) pour un nœud, afin de guider la sélection des nœuds dans l'arbre MCTS.

Args:
- node : Nœud pour lequel on calcule la valeur UCT.
- exploration_param : Paramètre de pondération qui contrôle le compromis entre exploration et exploitation.

Returns: 
- Une valeur numérique représentant l'attractivité du nœud pour la sélection.

### 6. Fonction select_best_node

- La fonction select_best_node utilise la valeur UCT pour sélectionner le meilleur enfant d'un nœud.

- Elle parcourt tous les enfants et retourne celui qui présente la valeur UCT la plus élevée, favorisant ainsi un bon compromis entre exploration et exploitation.

In [10]:
def select_best_node(node):
    return max(node.children, key=uct_value)


- Sélectionne le meilleur nœud parmi les enfants d'un nœud donné, en utilisant la valeur UCT.

Args:
- node : Nœud parent dont on souhaite choisir le meilleur enfant.

Returns: 
- Le nœud enfant ayant la valeur UCT la plus élevée.

### 7. Fonction simulate

- La fonction simulate exécute une série de simulations à partir d'un nœud donné.

- Elle commence par calculer le regret maximum et continue de poser des questions au décideur jusqu'à ce que le regret soit inférieur à un seuil epsilon.
- Elle utilise la fonction Query pour obtenir la préférence et la fonction Update pour ajuster le modèle en fonction des réponses du décideur.

In [11]:
def simulate(node, X, m, epsilon):
    alternative = node.state  # L'alternative pour laquelle on simule
    max_regret, x_star, y_star = mMR(X, m)
    
    while max_regret >= epsilon:
        answer = Query(x_star, y_star)
        m = Update(m, answer, x_star, y_star)
        max_regret, x_star, y_star = mMR(X, m)
    
    return max_regret


- Simule une série de décisions à partir d'un nœud pour explorer les conséquences de certaines alternatives.

Args:
- node : Nœud à partir duquel on effectue la simulation.
- X : Ensemble des alternatives.
- m : Modèle de programmation linéaire (MIP) pour les calculs.
- epsilon : Tolérance pour le regret.

Returns: 
- Le regret maximal calculé à la fin de la simulation.

### 8. Fonction backpropagate:

- La fonction backpropagate met à jour les statistiques d'un nœud et de ses ancêtres dans l'arbre après une simulation.

- Elle incrémente le compteur de visites et ajuste la valeur cumulée en fonction du résultat obtenu lors de la simulation.
- Cela permet de propager l'information à travers l'arbre, influençant les futures décisions.

In [12]:
def backpropagate(node, result):
    while node is not None:
        node.visits += 1
        node.value += result  # Mise à jour de la valeur (peut être le score ou regret)
        node = node.parent


- Met à jour les valeurs des nœuds parcourus après une simulation, en augmentant le nombre de visites et la valeur cumulative.

Args:
- node : Nœud à partir duquel on commence la rétropropagation.
- result : Résultat de la simulation (par exemple, le regret).

Returns: 
- Met à jour les statistiques des nœuds parcourus jusqu'à la racine.

### 9. Fonction mcts_search

- La fonction mcts_search exécute l'algorithme MCTS pour explorer l'espace des solutions.

- Elle effectue un nombre défini de simulations (num_simulations), sélectionnant des nœuds à explorer et en ajoutant de nouveaux nœuds si nécessaire.
- Après chaque simulation, elle met à jour l'arbre en utilisant les résultats pour influencer les décisions futures.
- À la fin, elle retourne l'état le plus visité, qui représente la meilleure alternative trouvée.

In [13]:
def mcts_search(root, X, m, epsilon, num_simulations=1000):
    for _ in range(num_simulations):
        node = root
        
        while node.children:
            node = select_best_node(node)
        
        if node.visits > 0:
            new_state = np.random.choice(X)  # Choisir un nouvel état ou alternative
            node = node.add_child(new_state)
        
        result = simulate(node, X, m, epsilon)
        backpropagate(node, result)
    
    return max(root.children, key=lambda n: n.visits).state


- Effectue la recherche MCTS pour explorer les alternatives et sélectionner la meilleure solution.

Args:
- root : Nœud racine de l'arbre de recherche.
- X : Ensemble des alternatives.
- m : Modèle de programmation linéaire (MIP) pour les calculs.
- epsilon : Tolérance pour le regret.
- num_simulations : Nombre de simulations à effectuer.

Returns
- L'alternative optimale trouvée après toutes les simulations.

### 10. Fonction vanilla_incremental_elicitation_with_mcts

- Cette fonction intègre MCTS dans l'algorithme d'élucidation incrémentale.

- Elle initialise un nœud racine pour MCTS et lance la recherche MCTS pour identifier la meilleure alternative parmi l'ensemble X.
- Le résultat final, c'est-à-dire l'alternative optimale recommandée, est affiché et retourné.

In [14]:
def vanilla_incremental_elicitation_with_mcts(X, m, epsilon):
    root = MCTSNode(state=None)  # Aucun état initial
    solution = mcts_search(root, X, m, epsilon)
    print(f"Solution finale recommandée avec MCTS: {solution}")
    return solution


- Lance l'algorithme d'élucidation incrémentale en utilisant MCTS pour trouver la meilleure alternative.

Args:
- X : Ensemble des alternatives.
- m : Modèle de programmation linéaire (MIP).
- epsilon : Tolérance pour le regret.

Returns:
- L'alternative optimale trouvée par MCTS.

### Exemple d'elicitation

In [15]:
def exemple_elicitation():
    X = np.array([[0.5, 0.2], [0.7, 0.1], [0.6, 0.3]])  # Ensemble de solutions
    m = Model()  # Initialisation du modèle MIP
    epsilon = 0.1  # Tolérance pour le regret minimax
    solution = vanilla_incremental_elicitation_with_mcts(X, m, epsilon)
    print(f"\nSolution finale recommandée: {solution}")
